In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sys, os, pickle

from tqdm.notebook import tqdm
from datetime import timedelta
#from utils import baseline_SCr

if os.getcwd()[-4:] == "code":
    os.chdir('../')

icu = './data/mimic-iv-2.2-parquet/icu/'
hosp = './data/mimic-iv-2.2-parquet/hosp/'

pd.set_option('mode.chained_assignment',  None) # 경고 off

In [2]:
labvalues = pd.read_csv('./data/labvalues/labvalues.csv')
icustays  = pd.read_parquet(icu+'icustays.parquet')

for i in labvalues.abbreviation.unique():
    globals()['chartevents_{}'.format(i)] = pd.read_parquet('./data/labvalues/chartevents_%s.parquet'%i)
    globals()['labevents_{}'.format(i)] = pd.read_parquet('./data/labvalues/labevents_%s.parquet'%i)

In [3]:
def resample(chartevents_,labevents_,icustays):
    rt = []
    tmp_itemid = chartevents_.itemid[0]

    icustays_intime = icustays[['subject_id','hadm_id','stay_id','intime']]
    icustays_intime['intime'] = pd.to_datetime(icustays_intime['intime'])
    icustays_intime = icustays_intime.rename(columns={'intime':'charttime'})

    icustays_outtime = icustays[['subject_id','hadm_id','stay_id','outtime']]
    icustays_outtime['outtime'] = pd.to_datetime(icustays_outtime['outtime'])
    icustays_outtime = icustays_outtime.rename(columns={'outtime':'charttime'})

    labevents_['charttime'] = pd.to_datetime(labevents_['charttime'])

    for i in tqdm(icustays.hadm_id.unique()):
        tmp_hosp = labevents_[labevents_['hadm_id']==i]
        tmp_hosp = tmp_hosp[['subject_id','hadm_id','charttime','itemid','valuenum']]

        for i in icustays[icustays['hadm_id']==i].stay_id.unique():
            tmp = chartevents_[chartevents_['stay_id']==i]
            tmp_id = icustays[icustays['stay_id']==i][['subject_id','hadm_id','stay_id']].iloc[0].to_list()
            tmp_intime = icustays_intime[icustays_intime['stay_id']==i]
            tmp_outtime = icustays_outtime[icustays_outtime['stay_id']==i]

            tmp = pd.concat([tmp, tmp_intime, tmp_outtime])

            tmp = tmp[(tmp['charttime'].values >= tmp_intime.charttime.values)&(tmp['charttime'].values <= tmp_outtime.charttime.values)]
            tmp.index = pd.DatetimeIndex(tmp['charttime'])

            tmp = pd.DataFrame(tmp['valuenum'].resample(rule='H', origin='start').last())
            tmp.reset_index(drop=False,inplace=True)
            tmp_lasttime = tmp['charttime'].values[-1]
            tmp = pd.concat([tmp,tmp_hosp])     
            tmp = tmp[(tmp['charttime'].values >= tmp_intime.charttime.values-np.timedelta64(1,'D'))&(tmp['charttime'].values <= tmp_lasttime)]
            tmp = tmp.sort_values(by='charttime',ascending=True)
            tmp = tmp.ffill()
            tmp = tmp[(tmp['charttime'].values >= tmp_intime.charttime.values)&(tmp['charttime'].values <= tmp_lasttime)]
            tmp[['subject_id','hadm_id','stay_id']] = tmp_id
            tmp['itemid'] = tmp_itemid
            rt.append(tmp)
    return pd.concat(rt)

In [6]:
labvalues.abbreviation.unique()

array(['Alb', 'Alk_Phos', 'AG', 'BUN', 'Ca', 'CK', 'D_Bil', 'Glu', 'HCT',
       'INR', 'PH', 'PHOS', 'Platelet', 'Cl', 'SCr', nan, 'Potassium',
       'T_Bil', 'WBC', 'Gl', 'Mg', 'Ca_ion', 'HCO3', 'AST', 'ALT', 'PTT',
       'baseexcess', 'lactate'], dtype=object)

In [7]:
for i in tqdm(labvalues.abbreviation.unique()):
    globals()['resample_{}'.format(i)] = resample(globals()['chartevents_{}'.format(i)],globals()['labevents_{}'.format(i)],icustays)
    if not os.path.isfile('./data/resample/resample_%s.parquet'%i):
        globals()['resample_{}'.format(i)].to_parquet('./data/resample/resample_%s.parquet'%i)

  0%|          | 0/28 [00:00<?, ?it/s]

  0%|          | 0/66239 [00:00<?, ?it/s]

  0%|          | 0/66239 [00:00<?, ?it/s]

  0%|          | 0/66239 [00:00<?, ?it/s]

  0%|          | 0/66239 [00:00<?, ?it/s]

...

  0%|          | 0/66239 [00:00<?, ?it/s]

  0%|          | 0/66239 [00:00<?, ?it/s]

  0%|          | 0/66239 [00:00<?, ?it/s]

  0%|          | 0/66239 [00:00<?, ?it/s]

  0%|          | 0/66239 [00:00<?, ?it/s]